# Test om koden er riktig implementert

Her er et forslag til testfunksjoner for å sjekke om koden er riktig implementert.
```assert variabel``` vil gi en feilmelding med mindre variabelen ```variabel = True```. For eksempel vil ```assert a == b``` gi en feilmelding med mindre ```a``` og ```b``` er like.

In [1]:
#For eksempel:
variable = True
assert variable, "You need to change 'variable' to True"

In [1]:
from layers import *
from neural_network import NeuralNetwork
from utils import onehot
import numpy as np

In [2]:
loss = CrossEntropy()

x = np.array([[0,1,4,3], [0,2,3,4]])
y = onehot(x, 5)

print(loss.forward(y,np.array([[0,1,4,3], [0,2,3,4]])))

0.0


In [7]:
#We choose some arbitrary values for the dimensions
b = 6
n_max = 7
m = 8
n = 5

d = 10
k = 5
p = 20

#Create an arbitrary dataset
x = np.random.randint(0, m, (b,n))
y = np.random.randint(0, m, (b,n_max))

#initialize the layers
feed_forward = FeedForward(d,p)
attention = Attention(d,k)
embed_pos = EmbedPosition(n_max,m,d)
un_embed = LinearLayer(d,m)
softmax = Softmax()
loss = CrossEntropy()

#a manual forward pass
X = onehot(x, m)
z0 = embed_pos.forward(X)
z1 = feed_forward.forward(z0)
z2 = attention.forward(z1)
z3 = un_embed.forward(z2)



#check the shapes
assert X.shape == (b,m,n), f"X.shape={X.shape}, expected {(b,m,n)}"
assert z0.shape == (b,d,n), f"z0.shape={z0.shape}, expected {(b,d,n)}"
assert z1.shape == (b,d,n), f"z1.shape={z1.shape}, expected {(b,d,n)}"
assert z2.shape == (b,d,n), f"z2.shape={z2.shape}, expected {(b,d,n)}"
assert z3.shape == (b,m,n), f"z3.shape={z3.shape}, expected {(b,m,n)}"
assert Z.shape == (b,m,n), f"Z.shape={Z.shape}, expected {(b,m,n)}"

#is X one-hot?
assert X.sum() == b*n, f"X.sum()={X.sum()}, expected {b*n}"


assert np.allclose(Z.sum(axis=1), 1), f"Z.sum(axis=1)={Z.sum(axis=1)}, expected {np.ones(b)}"
assert np.abs(Z.sum() - b*n) < 1e-5, f"Z.sum()={Z.sum()}, expected {b*n}"
assert np.all(Z>=0), f"Z={Z}, expected all entries to be non-negative"


In [5]:

#test the forward pass
x = np.random.randint(0, m, (b,n_max-1))
X = onehot(x, m)

#we test with a y that is shorter than the maximum length
n_y = n_max - 1
y = np.random.randint(0, m, (b,n_y))

#initialize a neural network based on the layers above
network = NeuralNetwork([embed_pos, feed_forward, attention, un_embed, softmax])
#and a loss function
loss = CrossEntropy()

#do a forward pass
Z = network.forward(X)

#compute the loss
L = loss.forward(Z, y)

#get the derivative of the loss wrt Z
grad_Z = loss.backward()

#and perform a backward pass
_ = network.backward(grad_Z)

#and and do a gradient descent step
_ = network.step_gd(0.01)

ValueError: operands could not be broadcast together with shapes (10,7) (10,6) (10,7) 

In [ ]:
"""
Here you may add additional tests to for example:

- Check if the ['d'] keys in the parameter dictionaries are not None, or receive something when running backward pass
- Check if the parameters change when you perform a gradient descent step
- Check if the loss decreases when you perform a gradient descent step

This is voluntary, but could be useful.
"""

"\nHere you may add additional tests to for example:\n\n- Check if the ['d'] keys in the parameter dictionaries are not None, or receive something when running backward pass\n- Check if the parameters change when you perform a gradient descent step\n- Check if the loss decreases when you perform a gradient descent step\n\nThis is voluntary, but could be useful.\n"

In [2]:
#check if loss is non-negative
assert L >= 0, f"L={L}, expected L>=0"
assert grad_Z.shape == Z.shape, f"grad_Z.shape={grad_Z.shape}, expected {Z.shape}"

#check if onehot(y) gives zero loss
Y = onehot(y, m)
L = loss.forward(Y, y)
assert L < 1e-5, f"L={L}, expected L<1e-5"


NameError: name 'L' is not defined

In [2]:
from layers import *
from neural_network import NeuralNetwork
from utils import onehot
import numpy as np

In [3]:
##Legger ved koden fra presentasjonen

from data_generators import get_train_test_addition

#dimensjoner og størrelser til x og y
n_digits = 2
n_max = 3*n_digits
m = 10
b = 6


#definerer størrelsen på parametermatrisene
d = 15
k = 5
p = 20

#henter treningsdata
data = get_train_test_addition(n_digits,samples_per_batch=100,n_batches_train=4)

In [4]:
from utils import onehot

x = data['x_train'][0]
X = onehot(x,m)
y = data['y_train'][0]
from neural_network import NeuralNetwork

#vi kan samle lagene i en liste som vi bruker for å
#initialisere et nevralt nettverk der vi kan bruke forward() og backward() 
#for å oppnå det samme som vi gjorde manuelt over

embed = EmbedPosition(n_max,m,d)
att1 = Attention(d,k)
ff1 = FeedForward(d,p)
un_embed = LinearLayer(d,m)
softmax = Softmax()
loss = CrossEntropy()

#Manuelt forward
z0 = embed.forward(X)
z1 = att1.forward(z0)
z2 = ff1.forward(z1)
z = un_embed.forward(z2)
Z = softmax.forward(z)



In [5]:
#evaluerer objektfunksjonen
L = loss.forward(Z,y)
print(L)

#finner den deriverte av objektfunksjonen mhp Z
dLdZ = loss.backward()

2.300504475938426


In [6]:
#Manuelt backwards
dLdZ = loss.backward()
dLdz = softmax.backward(dLdZ)
dLdz2 = un_embed.backward(dLdz)
dLdz1 = ff1.backward(dLdz2)
dLdz0 = att1.backward(dLdz1)
embed.backward(dLdz0)


layers = [embed,att1,ff1,un_embed,softmax]
nn = NeuralNetwork(layers)

Z = nn.forward(X)

#beregner loss med CrossEntropy
L = loss.forward(Z,y)
print(L)


2.300504475938426


In [7]:
xs = data['x_train']
ys = data['y_train']

n_batches = xs.shape[0]
n_iters = 100
step_size = 0.1

#treningsløkke tilsvarende algoritme 4 (med gradient descent)
for j in range(n_iters):
    losses = []
    for i in range(n_batches):
        x = xs[i]
        y = ys[i]

        X = onehot(x,m)
        Z = nn.forward(X)

        losses.append(loss.forward(Z,y))
        dLdZ = loss.backward()
        nn.backward(dLdZ)
        nn.step_gd(step_size)
    mean_loss = np.mean(losses)
    print("Iterasjon ", str(j), " L = ",mean_loss, "")

Iterasjon  0  L =  2.3028390127154177 
Iterasjon  1  L =  2.302829481581699 
Iterasjon  2  L =  2.302819949419086 


KeyboardInterrupt: 

In [8]:
xs = data['x_train']
ys = data['y_train']

n_batches = xs.shape[0]
n_iters = 100
step_size = 0.1

for j in range(n_iters):
    losses = []
    for i in range(n_batches):
        x = xs[i]
        y = ys[i]

        X = onehot(x,m)
        Z = nn.forward(X)

        losses.append(loss.forward(Z,y))
        dLdZ = loss.backward()
        nn.backward(dLdZ)
        nn.step_Adam()
    mean_loss = np.mean(losses)
    print("Iterasjon ", str(j), " L = ",mean_loss, "")

Iterasjon  0  L =  2.3025881555403434 
Iterasjon  1  L =  2.300733009084732 
Iterasjon  2  L =  2.2981905634975686 
Iterasjon  3  L =  2.2953586776968646 
Iterasjon  4  L =  2.292286311668885 
Iterasjon  5  L =  2.288926065627903 
Iterasjon  6  L =  2.2851826978563627 
Iterasjon  7  L =  2.2809027072110055 
Iterasjon  8  L =  2.2759420791191816 
Iterasjon  9  L =  2.270096076772378 
Iterasjon  10  L =  2.2632145432319652 
Iterasjon  11  L =  2.255120607249598 
Iterasjon  12  L =  2.245637234549224 
Iterasjon  13  L =  2.2346312123634395 
Iterasjon  14  L =  2.2220044773969247 
Iterasjon  15  L =  2.207711458733689 
Iterasjon  16  L =  2.19179843028377 
Iterasjon  17  L =  2.1745113166218544 
Iterasjon  18  L =  2.1563052639616105 
Iterasjon  19  L =  2.137855493315073 
Iterasjon  20  L =  2.119932669233894 
Iterasjon  21  L =  2.103355511169725 
Iterasjon  22  L =  2.0887980349317727 
Iterasjon  23  L =  2.0766444027419695 
Iterasjon  24  L =  2.0668956807486927 
Iterasjon  25  L =  2.